In [7]:
from IPython.display import display, clear_output
from urllib.request import urlopen
import requests as req
import pandas as pd
from datetime import datetime
import sched
import time
import json

s = sched.scheduler(time.time, time.sleep)

In [ ]:
def stock_table(stock_number_list):
    
    clear_output(wait=True)
    
    stock_num  = "|".join("tse_{}.tw".format(i) for i in stock_number_list)
    
    url = "http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=" + stock_num 
    r = req.get(url)
    info  = json.loads(r.text)
    info = info["msgArray"]
    
    info_index = ['c','n','z','tv','v','o','h','l','y']
    columns = ['股票代號','公司簡稱','當盤成交價','當盤成交量','累積成交量','開盤價','最高價','最低價','昨收價']
    
    
    stockinfo_table = pd.DataFrame(info, columns = info_index)
    stockinfo_table.columns = columns
    
    stockinfo_table = stockinfo_table.astype({'當盤成交價': "float", '當盤成交量':"int64", '累積成交量':"int64", '開盤價':"float", '最高價':"float", '最低價':"float",'昨收價':"float"})
    
    stockinfo_table["漲跌幅"] = (stockinfo_table["當盤成交價"] - stockinfo_table["昨收價"])/stockinfo_table["昨收價"] * 100
    
    now = datetime.now()
    
    print("更新時間: " + str(now.date()) +" "+ str(now.hour) + ":" +str(now.minute) + ":"+str(now.second))
    
    # 設定開始和結束時間
    start_time = datetime.strptime(str(now.date()) +" 9:30", "%Y-%m-%d %H:%M")
    end_time = datetime.strptime(str(now.date()) +" 13:30", "%Y-%m-%d %H:%M")
    
    s1 = stockinfo_table
    s1.insert(0,"公司",s1["股票代號"] + s1["公司簡稱"])
    s1 = s1.drop(["股票代號","公司簡稱"],axis =1)
    
    s1 = s1.set_index("公司")
    
    display(s1)
    
    if now > start_time and now < end_time:
        s.enter(5,0, stock_table, argument=(stock_number_list,))

In [ ]:
stock_list = ['1101','1102','1103','2330']

# 每秒定時器
s.enter(1, 0, stock_table, argument = (stock_list,))
s.run()

In [ ]:
# str轉換成datetime
datetime.strptime("2020-02-14 15:30", "%Y-%m-%d %H:%M")

In [ ]:
#str轉換成datetime
datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S")